
# Limpieza y transformación de datos (Notebook resuelto)
Dataset: `dataset_sucio.csv` (nulos, duplicados, outliers, fechas en texto y variables categóricas).  
Objetivo: preparar los datos para modelado sin entrar aún en EDA completo.


## 1. Carga del dataset y exploración inicial

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option("display.max_columns", None)

df = pd.read_csv("dataset_sucio.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

## 2. Detección y eliminación de duplicados

In [ ]:
duplicados_absolutos = df.duplicated().sum()
duplicados_absolutos

In [ ]:
df.duplicated().sum()

In [ ]:

duplicados_por_id = df.duplicated(subset=["id"]).sum()
print("Duplicados totales:", duplicados_absolutos)
print("Duplicados por id:", duplicados_por_id)


In [ ]:

# Eliminamos duplicados completos primero
df = df.drop_duplicates().copy()
# También nos aseguramos que cada 'id' sea único
df = df.drop_duplicates(subset=["id"]).copy()
df.shape


## 3. Detección y tratamiento de nulos

In [ ]:
df.isnull().sum()

In [ ]:

# Ejemplo: imputar nulos en 'ciudad' con 'Desconocido'
df["ciudad"] = df["ciudad"].fillna("Desconocido")

# Ejemplo: eliminar filas con fecha_contrato nula
df = df.dropna(subset=["fecha_contrato"])
df.isnull().sum()


## 4. Detección y manejo de outliers

In [ ]:

# Salario: revisar valores extremos
df["salario"].describe()


In [ ]:

# Podemos recortar valores de salario demasiado altos
q_low, q_high = df["salario"].quantile([0.01, 0.99])
df["salario"] = df["salario"].clip(lower=q_low, upper=q_high)
df["salario"].describe()


## 5. Transformaciones de columnas

In [ ]:

# Convertir columna fecha_contrato a datetime (invalid -> NaT)
df["fecha_contrato"] = pd.to_datetime(df["fecha_contrato"], errors="coerce")
df[["fecha_contrato"]].head()


In [ ]:

# Cambiar tipo de 'departamento' a categoría
df["departamento"] = df["departamento"].astype("category")
df.dtypes


In [ ]:

# Ejemplo extra 1: normalizar nombres a minúsculas
df["nombre"] = df["nombre"].str.lower()
df["apellido"] = df["apellido"].str.lower()
df[["nombre","apellido"]].head()


In [ ]:

# Ejemplo extra 2: crear columna año de la fecha
df["anio_contrato"] = df["fecha_contrato"].dt.year
df[["fecha_contrato","anio_contrato"]].head()


## 6. Binning de variables

In [ ]:

# Con pd.cut: agrupar edades
df["grupo_edad"] = pd.cut(df["edad"], bins=[17,29,49,69], labels=["joven","adulto","mayor"])
df[["edad","grupo_edad"]].head()


In [ ]:

# Con apply + lambda
df["rango_salario"] = df["salario"].apply(lambda x: "alto" if x>30000 else "bajo")
df[["salario","rango_salario"]].head()


## 7. Encoding de variables categóricas

In [ ]:

# Label encoding sencillo
df["departamento_code"] = df["departamento"].cat.codes
df[["departamento","departamento_code"]].head()


In [ ]:

# One-hot encoding
df_encoded = pd.get_dummies(df, columns=["ciudad"])
df_encoded.head()


## 8. Feature Engineering

In [ ]:

# Crear margen = ventas - costes
df["margen"] = df["ventas"] - df["costes"]

# Crear beneficio relativo
df["beneficio_pct"] = (df["margen"] / df["ventas"]).round(2)

# Crear columna nombre_completo
df["nombre_completo"] = df["nombre"] + " " + df["apellido"]

# Crear variable booleana
df["es_joven"] = df["edad"] < 30

# Extraer mes del contrato
df["mes_contrato"] = df["fecha_contrato"].dt.month

df.head()
